# Imports

In [ ]:
import os
import librosa
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt

# Tratamiento de la señal

In [ ]:
def audio_to_spectrogram(file_path, sr = None, n_fft = 2048, hop_length = 512):
    # Cargar el audio
    y, sr = librosa.load(file_path, sr = sr)
    
    # Calcular el espectrograma (STFT)
    S = librosa.stft(y, n_fft = n_fft, hop_length = hop_length)
    
    # Separar magnitud y fase (la magnitud es el espectrograma)
    magnitud, fase = np.abs(S), np.angle(S)
    
    return magnitud, sr, fase

def spectrogram_to_audio_con_fase(magnitud, fase, sr, hop_length = 512):
    # Combinar magnitud y fase para obtener el espectrograma complejo
    S_complejo = magnitud * np.exp(1j * fase)
    
    # Reconstruir el audio usando la ISTFT
    y_reconstruido = librosa.istft(S_complejo, hop_length=hop_length)
    
    return y_reconstruido

def spectrogram_to_audio_sin_fase(magnitud, sr, n_fft = 2048, hop_length = 512, n_iter = 32):
    # Reconstrucción del audio con Griffin-Lim
    audio_reconstruido = librosa.griffinlim(magnitud, n_iter=n_iter, hop_length=hop_length, n_fft=n_fft)
    
    return audio_reconstruido


In [ ]:
path = "path_carpeta"
audio_name = "nombre_audio"

# Generar el espectrograma
magnitud, sr, fase = audio_to_spectrogram(path + "//" + audio_name)

# Reconstruir el audio desde el espectrograma
audio_reconstruido = spectrogram_to_audio_con_fase(magnitud, fase, sr)
audio_reconstruido1 = spectrogram_to_audio_sin_fase(magnitud, sr)

# Guardar el audio reconstruido
sf.write("path_para_guardar_audio", audio_reconstruido, sr)
sf.write("path_para_guardar_audio", audio_reconstruido1, sr)



# Visualizar el espectrograma
plt.figure(figsize=(10, 6))
plt.imshow(librosa.amplitude_to_db(magnitud, ref=np.max), origin='lower', aspect='auto', cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title("Espectrograma")
plt.xlabel("Tiempo")
plt.ylabel("Frecuencia")
plt.tight_layout()
plt.show()